__<h2><center>ML Zoomcamp 2023 - Homework #5 </h2> </center>__

__Question 1__
Install Pipenv. 
What's the version of pipenv you installed?
Use --version to find out

>**Answer Q1:**  version 2023.10.3

__Question 2__
Use Pipenv to install Scikit-Learn version 1.3.1
What's the first hash for scikit-learn you get in Pipfile.lock?
Note: you should create an empty folder for homework and do it there.

__Models__
We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

features = ['job','duration', 'poutcome']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression().fit(X, y)
Note: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

DictVectorizer
LogisticRegression
With wget:

PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework
wget $PREFIX/model1.bin
wget $PREFIX/dv.bin

! pipenv install scikit-learn==1.3.1

! cat Pipfile.lock

>**Answer Q2:**  "sha256": "c34ce5877e8425f4fa78ed42bd54cc9d5c95ce77bcc811c00496d68c804f2986"


__Question 3__
Let's use these models!

Write a script for loading these models with pickle
Score this client:
{"job": "retired", "duration": 445, "poutcome": "success"}
What's the probability that this client will get a credit?

* 0.162
* 0.392
* 0.652
* 0.902

In [46]:
import pickle
with open('model1.bin', 'rb') as f_model:
    model = pickle.load(f_model)
with open('dv.bin', 'rb') as f_bin:
    dict_vectorizer = pickle.load(f_bin)

In [47]:
model, dict_vectorizer

(LogisticRegression(), DictVectorizer(sparse=False))

In [48]:
client = {"job": "retired", "duration": 445, "poutcome": "success"}
X = dict_vectorizer.transform(client)
model.predict_proba(X)[0, 1]

0.9019309332297606

__Question 4__
Now let's serve this model as a web service

Install Flask and gunicorn (or waitress, if you're on Windows)
Write Flask code for serving the model
Now score this client using requests:
url = "YOUR_URL"
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
requests.post(url, json=client).json()
What's the probability that this client will get a credit?

* 0.140
* 0.440
* 0.645
* 0.845

In [49]:
! pipenv install flask gunicorn

Installing flask...
Resolving flask...
✔ Installation Succeeded
⠋ Installing flask...
Installing gunicorn...
Resolving gunicorn...
✔ Installation Succeeded
⠋ Installing gunicorn...
Installing dependencies from Pipfile.lock (4f2986)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


Be sure that the wevservice is running:

uvicorn server:app --reload

In [50]:
import requests

url = "http://127.0.0.1:8000/predict"
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
response = requests.post(url, json=client)
result = response.json()
print(result)


{'prediction': 0.13968947052356817}


>**Answer Q4:** 0.140

__Docker__
Install Docker. We will use it for the next two questions.

For these questions, we prepared a base image: svizor/zoomcamp-model:3.10.12-slim. You'll need to use it (see Question 5 for an example).

This image is based on python:3.10.12-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

FROM python:3.10.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
We already built it and then pushed it to svizor/zoomcamp-model:3.10.12-slim.

Note: You don't need to build this docker image, it's just for your reference.

__Question 5__
Download the base image svizor/zoomcamp-model:3.10.12-slim. You can easily make it by using docker pull command.

So what's the size of this base image?

* 47 MB
* 147 MB
* 374 MB
* 574 MB

You can get this information when running docker images - it'll be in the "SIZE" column.

In [51]:
! docker pull svizor/zoomcamp-model:3.10.12-slim

3.10.12-slim: Pulling from svizor/zoomcamp-model
Digest: sha256:e8441100b9d8da56344f50c673eb2daded3c61ce9565e45c3592c02f34fb3149
Status: Image is up to date for svizor/zoomcamp-model:3.10.12-slim
docker.io/svizor/zoomcamp-model:3.10.12-slim

What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview svizor/zoomcamp-model:3.10.12-slim


In [60]:
! docker image inspect svizor/zoomcamp-model:3.10.12-slim | jq '.[0].Size'

147133320


**Answer Q5:** 147 MB

__Dockerfile__
Now create your own Dockerfile based on the image we prepared.

It should start like that:

FROM svizor/zoomcamp-model:3.10.12-slim
Now complete it:

Install all the dependencies form the Pipenv file
Copy your Flask script
Run it with Gunicorn
After that, you can build your docker image.

__Question 6__
Let's run your docker container!

After running it, score this client once again:

url = "YOUR_URL"
client = {"job": "retired", "duration": 445, "poutcome": "success"}
requests.post(url, json=client).json()
What's the probability that this client will get a credit now?

* 0.168
* 0.530
* 0.730
* 0.968

! docker build -t my-fastapi-app .

In [65]:
! docker run -d -p 8000:8000 my-fastapi-app

77190ca35eb7b4925e02731bd90fc67000df5b42ef3ccc2e74594962a58804b3
docker: Error response from daemon: driver failed programming external connectivity on endpoint funny_kowalevski (04b18004c7bce318aee0624be3450e2318eed4528180683a7d55b12fd8806954): Bind for 0.0.0.0:8000 failed: port is already allocated.


In [69]:
url = "http://localhost:8000/predict"
client = {"job": "retired", "duration": 445, "poutcome": "success"}
answer = requests.post(url, json=client).json()
print(answer)

{'prediction': 0.726936946355423}


>**Answer Q6:** 0.730